In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelBinarizer
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
from sklearn.model_selection import GroupKFold
import warnings
warnings.simplefilter('ignore')

df = pd.read_csv("train.csv") 
df_test = pd.read_csv("test.csv") 

#df = df.drop(["Country","City"], axis=1)
#df_test = df_test.drop(["Country","City"], axis=1)

df['date'] = pd.to_datetime({'year': df['year'], 'month': df['month'], 'day': df['day']})
df_test['date'] = pd.to_datetime({'year': df_test['year'], 'month': df_test['month'], 'day': df_test['day']})

df = df.drop(["year","month","day"], axis=1)
df_test = df_test.drop(["year","month","day"], axis=1)

#df = df.sort_values(['date', 'Country','City']).reset_index(drop=True)
#df_test = df_test.sort_values([ 'date','Country','City']).reset_index(drop=True)

df_id = df.copy()
df_test_id = df_test.copy()

df = df.drop(["date","id"], axis=1)
df_test = df_test.drop(["date","id"], axis=1)

df

,Country,City,lat,lon,co_cnt,co_min,co_mid,co_max,co_var,o3_cnt,...,ws_min,ws_mid,ws_max,ws_var,dew_cnt,dew_min,dew_mid,dew_max,dew_var,pm25_mid
0,Australia,Brisbane,-27.46794,153.02809,38,0.749,2.590,2.633,0.850,29,...,0.241,1.088,3.101,1.983,17,7.671,10.358,15.112,13.424,19.901
1,Australia,Darwin,-12.46113,130.84185,47,2.594,3.181,4.828,1.208,49,...,0.828,3.473,7.396,10.411,62,21.324,23.813,24.221,2.021,13.741
2,Australia,Melbourne,-37.81400,144.96332,17,1.190,1.197,2.200,0.248,123,...,0.000,2.107,8.089,15.719,22,10.309,13.133,15.422,6.355,25.918
3,Australia,Newcastle,-32.92953,151.78010,63,4.586,11.044,14.802,24.186,90,...,0.284,0.503,3.592,2.485,116,7.146,10.685,13.344,9.417,174.370
4,Australia,Perth,-31.95224,115.86140,47,4.689,8.681,11.100,10.011,83,...,0.500,0.755,3.396,1.937,93,1.091,3.277,12.272,4.109,167.063
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195936,United States,Jacksonville,30.33218,-81.65565,12,0.694,0.995,1.301,0.090,26,...,2.195,2.710,6.125,3.757,12,16.774,22.679,26.058,13.252,16.150
195937,United States,Las Vegas,36.17497,-115.13722,14,0.528,1.256,3.226,1.743,8,...,1.002,2.974,6.861,8.354,12,10.432,14.741,15.827,7.078,16.895
195938,United States,Milwaukee,43.03890,-87.90647,171,1.975,6.627,6.639,5.293,112,...,0.994,1.087,2.578,0.612,26,2.049,3.531,6.686,5.286,86.299
195939,Vietnam,Hanoi,21.02450,105.84117,31,2.613,2.704,8.767,4.317,108,...,1.005,3.058,6.005,6.085,51,1.922,7.443,7.716,4.642,36.523


In [2]:
cat_cols = ["Country","City"]
num_cols = ['lat', 'lon', 'co_cnt', 'co_min', 'co_mid', 'co_max',
       'co_var', 'o3_cnt', 'o3_min', 'o3_mid', 'o3_max', 'o3_var', 'so2_cnt',
       'so2_min', 'so2_mid', 'so2_max', 'so2_var', 'no2_cnt', 'no2_min',
       'no2_mid', 'no2_max', 'no2_var', 'temperature_cnt', 'temperature_min',
       'temperature_mid', 'temperature_max', 'temperature_var', 'humidity_cnt',
       'humidity_min', 'humidity_mid', 'humidity_max', 'humidity_var',
       'pressure_cnt', 'pressure_min', 'pressure_mid', 'pressure_max',
       'pressure_var', 'ws_cnt', 'ws_min', 'ws_mid', 'ws_max', 'ws_var',
       'dew_cnt', 'dew_min', 'dew_mid', 'dew_max', 'dew_var']

# 23.17
from sklearn.preprocessing import StandardScaler # 23.13
from sklearn.preprocessing import MinMaxScaler #23.15
from sklearn.preprocessing import RobustScaler # 23.06
from sklearn.preprocessing import QuantileTransformer # 23.07

#scaler = QuantileTransformer(
   # n_quantiles=100,
   # random_state=42,
    #output_distribution='normal'
#)

scaler = RobustScaler(quantile_range=(25.0, 75.0))
#scaler = StandardScaler()
scaler.fit(df[num_cols])
df[num_cols] = scaler.transform(df[num_cols])
df_test[num_cols] = scaler.transform(df_test[num_cols])


for col in cat_cols:
    encoder = df[col].value_counts()
    df[f'label_{col}'] = df[col].map(encoder)
    df_test[f'label_{col}'] = df_test[col].map(encoder)

In [3]:
le = LabelEncoder()
df["Country"] = le.fit_transform(df["Country"])
df["City"] = le.fit_transform(df["City"])
df_test["Country"] = le.fit_transform(df_test["Country"])
df_test["City"] = le.fit_transform(df_test["City"])


In [4]:
# 目的変数と説明変数の定義
X = df.drop(columns=['pm25_mid'])
y = df['pm25_mid']

# train_test_splitのインポート
#from sklearn.model_selection import train_test_split

# shuffle=Falseとすることで時系列が混ざるのを防止

# 学習データおよび検証データと、評価データに80:20の割合で2分割する
#X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, test_size=0.20, shuffle=False)


print("finish")

finish


In [5]:
import numpy as np
import pandas as pd

# グラフ描画用
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
sns.set_style('darkgrid')

import lightgbm as lgb

from sklearn.datasets import make_classification
from sklearn.metrics import roc_auc_score, roc_curve, accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
import xgboost as xgb
from sklearn.model_selection import GroupKFold

In [7]:
def xgboost(X_train, Y_train, X_valid, Y_valid, X_test):
    xgb_params = {
      'objective': 'reg:linear',
      'eval_metric': 'rmse',
      # "verbosity": 0,
      "seed": 2022,
      "eta": 0.01,
      "num_boost_round": 10000,
      # "early_stopping_rounds": 10,
      # "verbose_eval": 100,
    }

    lgb_train = xgb.DMatrix(X_train, label=Y_train)
    lgb_valid = xgb.DMatrix(X_valid, label=Y_valid)
    lgb_test = xgb.DMatrix(X_test)
    evals = [(lgb_train, 'train'), (lgb_valid, 'eval')]
    evals_result = {}
    model = xgb.train(xgb_params,
                    lgb_train,
                    evals=evals,
                    evals_result=evals_result,
                    num_boost_round=10000,
                    early_stopping_rounds=10,
                    verbose_eval=1000)
  
    # 検証データに対する予測値を求める
    va_pred = model.predict(lgb_valid)

    #テストデータに対する予測値を求める
    te_pred = list(model.predict(lgb_test))

    return va_pred, te_pred, model

va_pred3, te_pred3, model = xgboost(X_train, y_train, X_test, y_test, df_test)

score = np.sqrt(mean_squared_error(y_test, va_pred3))
print(f"Validation RMSE score : {score:.4f}")
print()
print(f"Test data best socre : {19.1648:.4f}")
va_pred3 = pd.Series(va_pred3)
va_pred3.to_csv("XGBoost_test.csv",index=None,header=None)

NameError: name 'X_test' is not defined

In [11]:
# 学習用のパラメータ
xgb_params = {
      'objective': 'reg:linear',
      'eval_metric': 'rmse',
      # "verbosity": 0,
      "seed": 2022,
      "eta": 0.01,
      "num_boost_round": 10000,
      # "early_stopping_rounds": 10,
      # "verbose_eval": 100,
    }

FOLD = 5
NUM_ROUND = 100
VERBOSE_EVAL = -1


group = X['City']
scores = []
valid_pred = []
models = []
kf = GroupKFold(n_splits=FOLD)

for fold, (train_indices, valid_indices) in enumerate(kf.split(X, y, group)):
    X_train, X_valid = X.iloc[train_indices], X.iloc[valid_indices]
    y_train, y_valid = y.iloc[train_indices], y.iloc[valid_indices]
    lgb_train = xgb.DMatrix(X_train, y_train)
    lgb_valid = xgb.DMatrix(X_valid, y_valid)
    lgb_test = xgb.DMatrix(df_test)
    evals = [(lgb_train, 'train'), (lgb_valid, 'eval')]
    evals_result = {}

    # model の作成、学習
    model = xgb.train(xgb_params,
                    lgb_train,
                    evals=evals,
                    evals_result=evals_result,
                    num_boost_round=10000,
                    early_stopping_rounds=10,
                    verbose_eval=1000)

    # 検証データに対する予測値を求める
    va_pred = model.predict(lgb_valid)
    score = np.sqrt(mean_squared_error(y_valid, va_pred))
    print(f" RMSE score : {score:.4f}")
    scores.append(score)

    #テストデータに対する予測値を求める
    te_pred = list(model.predict(lgb_test))
    
cv_score = np.mean(scores)
print()
print("==="*20)
print(f'CV score: {cv_score}')
print()
print("best socre : 22.365195931058942")
#val_pred = pd.Series(valid_pred)
#val_pred.to_csv("XGBT_test.csv",index=None,header=None)
#print(len(val_pred[0])+len(val_pred[1])+len(val_pred[2])+len(val_pred[3])+len(val_pred[4]))

[13:28:55] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.6.0/src/objective/regression_obj.cu:203: reg:linear is now deprecated in favor of reg:squarederror.
[13:28:55] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "num_boost_round" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[0]	train-rmse:68.39771	eval-rmse:74.38461
[1000]	train-rmse:19.32927	eval-rmse:24.70061
[1031]	train-rmse:19.28275	eval-rmse:24.68633
 RMSE score : 24.6855
[13:30:23] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.6.0/src/objective/regression_obj.cu:203: reg:linear is now deprecated in favor of reg:squarederror.
[13:30:23] WARNING: C:/Users/Administrator/workspace/xgboost-win6

In [ ]:
sample = pd.read_csv("submit_sample.csv",header=None) 

#pred = model.predict(df_test)
#pred = pd.Series(pred)
pred_df = pd.DataFrame(te_pred3,columns=['pred'])

pred_df["id"] = df_test_id["id"]
pred_df = pred_df.sort_values(['id']).reset_index(drop=True)
sample[1] = pred_df["pred"]
sample.to_csv("XGboost_sabmission.csv",index=None,header=None)
sample